In [1]:
import tensorflow as tf 
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

2025-12-26 17:51:19.619655: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-26 17:51:19.706453: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-26 17:51:19.706541: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-26 17:51:19.709233: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-26 17:51:19.722843: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-26 17:51:19.725394: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
## Load the trained model.scaler,onehot_encoder
model=load_model('model.h5')

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [9]:
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [10]:
geo_encoded=onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_enoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

/home/rguktrkvalley/ANN Classification/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [11]:
input_df=pd.DataFrame([input_data])
input_df=input_df.drop('Geography',axis=1)
input_df['Gender']=label_encoder_gender.fit_transform(input_df['Gender'])

input_final_df=pd.concat([input_df,geo_enoded_df],axis=1)

In [13]:
input_final_df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,0,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [14]:
input_scaled=scaler.transform(input_final_df)

In [15]:
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 395ms/step


array([[0.06435982]], dtype=float32)

In [18]:
predication_prob=prediction[0][0]
if predication_prob>0.5:
    print(f'The customer will stay in the bank with a probability of {predication_prob}')
else:
    print(f'The customer will leave the bank with a probability of {predication_prob}')

The customer will leave the bank with a probability of 0.06435982137918472
